# PRAC 01

## Launch CASA
To start CASA, open a unix terminal and run the command
```
$ casa # casapy will also work

```

This will launch a CASA shell, and a CASA logger. The shell is the environment from which CASA runs and the logger is for logging CASA processes. Bellow is a screenshot of the CASA shell (left) and logger (right)

![CASA shell and logger](figures/launching_casa_ss.png)


## CASA Tasks
CASA processes are organized into tasks and tools. We will only be using CASA tasks in course. 

**NOTE**: Commands that are executed on the command line will be preceeded by the "$" symbol and commands exucted in the casa shell wil be preceeded by the ">" symbol. 

## The Measurement Set

Visibility data is generally stored in a format called a *CASA Table*, and a CASA table containing visibility data is called a *Measurement set* (MS/ms). This is a directory (or database) which holds the visibility data and other information related an observation.

You are provided a with an MS from a KAT-7 simulation. Find the MS [here](link to ms). Run `ls` to see the contents fo the MS

```
$ ls <path to MS>
ANTENNA/ DATA_DESCRIPTION/ FEED/ FIELD/ FLAG_CMD/ HISTORY/ OBSERVATION/ POINTING/ POLARIZATION/ PROCESSOR/   SORTED_TABLE/ SOURCE/ SPECTRAL_WINDOW/ STATE/ table.dat table.f0 table.f1 table.f2 table.f2_TSM1 table.f3 table.f3_TSM1 table.f4 table.f4_TSM1 table.f5 table.f5_TSM1 table.f6 table.f6_TSM1 table.f7 table.f7_TSM1 table.f8 table.info table.lock
```

The MS contains sub directories, which are themselves CASA tables. For example,
```
$ ls <Path to MS>/ANTENNA  
table.dat  table.f0  table.info  table.lock
```

as well *table.\** files which hold the table data. 

In CASA, the `ms` tool is used to interface and manipulate the MS. But before we proceed, lets use the `browsetable` task view the MS in a GUI

```
> MSName = <Path to MS> # For convinience, assign MS name to vairiable
> browsetable(MSName)
```

A window like the one below will pop up
![CASA shell and logger](figures/browse_table_ms.png)

As can be seen, the MS data is organized into different fields (columns). For example, the *UVW* column holds the *u,v,w* coordinates of all the visbilites, and the *DATA* column has the corresponding data. That is, the *i*th row of *DATA* has the data for the *i*th coordinate of *UVW*. You will learn more about the other MS fields as the course progresses. 

## CASA Tasks

Tasks utilize a number of tools and accomplish a specific data analysis or calibration goal. To get a list of the tasks run:

`> tasklist`  
or,

`> taskhelp`  

For a brief summary of the tasks. For more extensive information about any of the task run:

`> help <task name>`

For example, to get help about the `tasklist` task simply run:

`> help tasklist`

![help example](figures/help_tasklist.png)

## Task execution
There are two ways of executing CASA tasks. The first, "interactive", the first is to list the task parameters and set them one by one. And, the second, "non-interactive",  is by executing the tasks as you would any other python function. For example, the *listobs* task gives a summary information about an MS. This task can be executed interactively like this:

Note: Find the MS [here](link to MS). Then assign it to the variable *MSName*
```
> MSName = <path to MS>
```

 * First view the task parameters using the *inp* command  

```
> inp listobs
```
![inp](figures/inp_listobs.png)


 * Then set the parameters  

```
> vis = MSName
```

 * Execute the task  

```
> go
```
![listobs](figures/list_obs.png)


Now, lets go over some data inspection and visualisation tasks. We will also use CASA to flag bad data points from the data.


### plotants
The `plotants` task plots the antenna layout of the observing instrument in the local coordinate system (east, north, up)

```
> inp plotants
> vis = <MS name>
> go
```
![plotants](figures/plotants_kat7.png)


### plotuv
Similarly, the `plotuv` task plots the uv-coverage of the observation. Note that the data has 3 fields, the that the scientist wants to study (target), as well as two calibrator fields. The one calibrator field is for calibrating the bandpass of the instrument durring the observation, while the other is calibrator the complex antenna gains. These details of how these fields are used will be disccused later in the course. For now, just note that the uv-coverage of the three fields is different as they are at different parts of the sky.

```
> inp plotuv
> vis = <MS name>
> field = <field>
> go
```

| field 0 (Bandpass cal)| field 1 (Target) | field 2 (Gain cal) |
|:---:|:-----------------:|:----------------------:|  
| ![plotants](figures/uvcov_0.png) |![plotants](figures/uvcov_1.png) |![plotants](figures/uvcov_2.png)| 

### plotms
The plotms task is one of the most useful tasks in CASA. It allows the user to visualise the data in many different ways, which is a critical step in devising a good data reduction strategy. 

Launch the the task
```
> inp plotms
> vis = <MS name>
> correlation = "xx" # Choose the xx correlation to simplify things
> go
```

By default, plotms will plot the visibility amplitude as a function time. After careful inspection, plot the amplitude as a function of channel.

## Flagging 'bad' data

In this section, we will use plotms to identify bad data points. We will then use the flagdata task to flag these data in the MS.

Lets plot the amplitude as a function of time
```
> plotms(vis=msfile, xaxis = 'channel', yaxis='amp', avgbaseline=T, yselfscale=T, coloraxis='field')
```

From this I decide to flag all data with an amplitude greater than 1.0. The tool I use to do this flagdata

```
> clipvals=[0.0,1]
> flagdata(vis=msfile, mode='clip', clipminmax=clipvals, flagbackup=False)
```
![scan 15 lines](figures/ampVtime_after_clip.png)



The data still does not look good. Let us now plot the amplitude as a function of channel. To simplify things, we only select one of the fields (field 1)

```
> plotms(vis=msfile, xaxis = 'channel', yaxis='amp', avgbaseline=T, yselfscale=T, avgtime='10000', field='1')
```

I identify channels 204 and 212 as being particularly bad and flag them.
![bad lines before](figures/ampVchannel_before_line1.png)

```
badchans = '0:204~204,0:212~212'
flagdata(vis=msfile, spw=badchans, flagbackup=False)
```
![bad lines after](figures/ampVchannel_after_line1.png)


Lets go back our amplitude vs time plot again. 
```
plotms(vis=msfile, xaxis = 'time', yaxis='amp', avgbaseline=T, yselfscale=T, coloraxis='field' )
```

We can see that scan 14, ad 15 are particularly bad. Lets make amplitude vs channel plots, but only selecting one of these channels, then the other.


```
> plotms(vis=msfile, xaxis = 'channel', yaxis='amp', avgbaseline=T, yselfscale=T, avgtime='10000', scan='14')
```

From this plot, I identify channels 113, 313, 369, 597, 599, 601-607 as needing to be flagged
![scan 14 lines](figures/ampVchannel_before_line2.png)

```
badchans = '0:113~113,0:313~313,0:369~369,0:599~599,0:597~597,0:601~607'
flagdata(vis=msfile, spw=badchans, flagbackup=False)
```
![scan 14 lines](figures/ampVchannel_after_line2.png)

Similarly, selecting scan 15, we see that channels 594-600 also need to be flagged.
![scan 15 lines](figures/ampVchannel_before_line3.png)



Going back to the amplitude vs time plot, we see that data is much cleaner.

![flagged data](figures/after_scan15_channel_flags.png)



